# Marketing Analytics

An online retail business, is facing reduced customer engagement and conversion rates despite launching several new online marketing campaigns. They are reaching out to you to help conduct a detailed analysis and identify areas for improvement in their marketing strategies.


#### Problems : 
1. Reduced Customer Engagement: The number of customer interactions and engagement with the site and marketing content has declined.
2. Decreased Conversion Rates: Fewer site visitors are converting into paying customers.
3.  High Marketing Expenses: Significant investments in marketing campaigns are not yielding expected returns.
4. Need for Customer Feedback Analysis: Understanding customer opinions about products and services is crucial for improving engagement and conversions.


In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import pyodbc


### Set Up the Connection to SQL Server

In [2]:
# Define connection parameters
server = 'V_M_SUTHAR\\SQLEXPRESS;'
database = 'Marketing'

# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes"
conn = pyodbc.connect(connection_string)

# Test the connection
print("Connected to the database.")

Connected to the database.


### Load the Datasets into Pandas DataFrames

In [3]:
# Example: Load customer data
customers_query = "SELECT * FROM dbo.customers;"
customers_df = pd.read_sql(customers_query, conn)

# Example: Load geography data
geography_query = "SELECT * FROM dbo.geography;"
geography_df = pd.read_sql(geography_query, conn)

# Example: Load other datasets as needed
reviews_query = "SELECT * FROM dbo.customer_reviews;"
reviews_df = pd.read_sql(reviews_query, conn)

C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\2751482802.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers_df = pd.read_sql(customers_query, conn)
C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\2751482802.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  geography_df = pd.read_sql(geography_query, conn)
C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\2751482802.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews_df = pd.read_sql(reviews_query, conn)


### Perform Data Processing and Cleaning in Python

a. Join customers and geography

In [4]:
# Left join customers with geography to enrich customer data with geographic information
customers_geography_df = customers_df.merge(
    geography_df, how='left', left_on='GeographyID', right_on='GeographyID'
)

b. Categorize Products Based on Price

In [5]:
# Add a price category
products_query = "SELECT * FROM dbo.products;"
products_df = pd.read_sql(products_query, conn)

products_df['PriceCategory'] = pd.cut(
    products_df['Price'],
    bins=[-float('inf'), 50, 200, float('inf')],
    labels=['Low', 'Medium', 'High']
)

C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\2528191736.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products_df = pd.read_sql(products_query, conn)


c. Clean Whitespace in ReviewText

In [6]:
reviews_df['ReviewText'] = reviews_df['ReviewText'].str.replace('  ', ' ')

d. Normalize engagement_data

In [7]:
engagement_query = "SELECT * FROM dbo.engagement_data;"
engagement_df = pd.read_sql(engagement_query, conn)

# Normalize ContentType
engagement_df['ContentType'] = engagement_df['ContentType'].str.replace(
    'Socialmedia', 'Social Media'
).str.upper()

# Split ViewsClicksCombined into Views and Clicks
engagement_df[['Views', 'Clicks']] = engagement_df['ViewsClicksCombined'].str.split('-', expand=True)

# Format EngagementDate
engagement_df['EngagementDate'] = pd.to_datetime(engagement_df['EngagementDate']).dt.strftime('%d.%m.%Y')

C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\3222990855.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  engagement_df = pd.read_sql(engagement_query, conn)


e. Identify and Handle Duplicate Records in customer_journey

In [8]:
journey_query = "SELECT * FROM dbo.customer_journey;"
journey_df = pd.read_sql(journey_query, conn)

# Add a row number column to identify duplicates
journey_df['row_num'] = journey_df.groupby(
    ['CustomerID', 'ProductID', 'VisitDate', 'Stage', 'Action']
).cumcount() + 1

# Remove duplicates, keeping the first occurrence
cleaned_journey_df = journey_df[journey_df['row_num'] == 1].drop(columns=['row_num'])

# Fill missing durations with average duration per date
cleaned_journey_df['Duration'] = cleaned_journey_df['Duration'].fillna(
    cleaned_journey_df.groupby('VisitDate')['Duration'].transform('mean')
)

C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\2463068171.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  journey_df = pd.read_sql(journey_query, conn)


### Save the Cleaned Data Back to SQL Server

In [9]:
from sqlalchemy import create_engine

# Define parameters
server = 'V_M_SUTHAR\\SQLEXPRESS'
database = 'Marketing'

# Create engine with the correct driver
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

# Save cleaned DataFrame
try:
    cleaned_journey_df.to_sql('cleaned_customer_journey', engine, if_exists='replace', index=False)
    print("DataFrame successfully saved to SQL Server.")
except Exception as e:
    print(f"Error: {e}")


DataFrame successfully saved to SQL Server.


### Export the Data to Files

In [26]:
engagement_df.to_csv('Cleaned Data/cleaned_engagement.csv', index=False)
reviews_df.to_csv('Cleaned Data/cleaned_reviews.csv', index=False)
products_df.to_csv('Cleaned Data/cleaned_product.csv', index=False)
customers_geography_df.to_csv('Cleaned Data/cleaned_customer_geography.csv', index=False)
cleaned_journey_df.to_csv('Cleaned Data/cleaned_customer_journey.csv', index=False)
# cleaned_journey_df.to_excel('Cleaned Data/cleaned_customer_journey.xlsx', index=False)

## Customer Reviews Enrichment

In [11]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [12]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\viraj\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [13]:
# Define a function to fetch data from a SQL database using a SQL query
def fetch_data_from_sql():

    conn_str = (
        "Driver={SQL Server};"  
        "Server=V_M_SUTHAR\\SQLEXPRESS;"  
        "Database=Marketing;"  
        "Trusted_Connection=yes;" 
    )
    # Establish the connection to the database
    conn = pyodbc.connect(conn_str)
    
    # Define the SQL query to fetch customer reviews data
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"
    
    df = pd.read_sql(query, conn)
    
    conn.close()
    
    # Return the fetched data as a DataFrame
    return df

In [14]:
# Fetch the customer reviews data from the SQL database
customer_reviews_df = fetch_data_from_sql()

C:\Users\viraj\AppData\Local\Temp\ipykernel_10424\2708001989.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [15]:
# Initialize the VADER sentiment intensity analyzer for analyzing the sentiment of text data
sia = SentimentIntensityAnalyzer()

In [16]:
# Define a function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    return sentiment['compound']

In [17]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):

    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  
        elif rating == 3:
            return 'Mixed Positive' 
        else:
            return 'Mixed Negative'  
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  
        elif rating == 3:
            return 'Mixed Negative'  
        else:
            return 'Mixed Positive'  
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  
        elif rating <= 2:
            return 'Negative'  
        else:
            return 'Neutral'  

In [18]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0' 
    else:
        return '-1.0 to -0.5' 

In [19]:
# Apply sentiment analysis to calculate sentiment scores for each review
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [20]:
# Apply sentiment categorization using both text and rating
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)


In [21]:
# Apply sentiment bucketing to categorize scores into defined ranges
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [22]:
# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets
print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [24]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
customer_reviews_df.to_csv('Cleaned Data/reviews_with_sentiment_new.csv', index=False)
